In [53]:
# import necessary libaries and data files
import numpy as np
from data import datas
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
#load data and manupulate datas if needed
bus_data, line_data, load_data, gen_data, wind_data = datas() #data loading

line_data = pd.DataFrame(line_data).sort_values(by=[0,1]).values #line data sorting and data type change

bus_data = pd.DataFrame(bus_data).sort_values(by = [0]).values #Bus data sorting and data type change

load_data = pd.DataFrame(load_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Load data sorting and data type change

gen_data = pd.DataFrame(gen_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Generation data sorting and data type change

wind_data = pd.DataFrame(wind_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Wind Generation data sorting and data type change

In [55]:
base_mva = 100
R = line_data[:, 2]
X = line_data[:, 3]
B = 0.5j * line_data[:, 4]
taps = line_data[:,5]
Bsh = 1j* bus_data[:, 3]
Z = R + 1j * X
Y = 1 / Z
nline = len(line_data[:, 0])
nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses

C:\Users\bpanthee\AppData\Local\Temp\ipykernel_21256\2570250330.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses


In [56]:
def tap_handle(x):
    if np.real(x) == 0:
        return 1
    else:
        return complex(x)
    
# Y Bus Formation
def Y_bus():
    Ybus = np.zeros((nbus, nbus), dtype=complex)
    Y1 = np.zeros((73, 73), dtype=complex)
    np.fill_diagonal(Y1,Bsh)
    
    for k in range(nline):
        # Off Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 1]) - 1] -= (Y[k])/np.conj(tap_handle(taps[k]))
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 0]) - 1] -= (Y[k])/tap_handle(taps[k])

        # Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 0]) - 1] += (Y[k] + B[k])/pow(abs(tap_handle(taps[k])),2)
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 1]) - 1] += Y[k] + B[k]
    
    Y_bus_ = Ybus + Y1
    return Y_bus_

In [57]:
#From Bus Data
bus_no = bus_data[:, 0].astype(int)
bus_type = bus_data[:,1].astype(int)

#From Load Data
pl = np.zeros(nbus)
pf = np.zeros(nbus)
delta = np.zeros(nbus)
for k in load_data.index:
    pl[k-1] = load_data.loc[k, 2]/base_mva #base power
    pf[k-1] = load_data.loc[k, 1]
ql = pl * np.tan(np.arccos(pf))

#From Generation Data
pg_ = np.zeros(nbus)
qg = np.zeros(nbus)
vmag = np.ones(nbus)
qmin = np.zeros(nbus)
qmax = np.zeros(nbus)
for k in gen_data.index:
    pg_[k-1] = (gen_data.loc[k, 4]).sum()/base_mva
    vmag[k-1] = np.mean(gen_data.loc[k, 3])
    qmin[k-1] = gen_data.loc[k, 2].sum()
    qmax[k-1] = gen_data.loc[k, 1].sum()

# From Wind Data
pwg = np.zeros(nbus)
for k in wind_data.index:
    pwg[k-1] = (wind_data.loc[k, 4])/base_mva

pg = pg_ + pwg
b_data = pd.DataFrame()
b_data['Bus_No'] = bus_no
b_data['Bus_type'] = bus_type
b_data['Pg'] = pg
b_data['Qg'] = qg
b_data['Pl'] = pl
b_data['Ql'] = ql
b_data['Vmag'] = vmag
b_data['Delta'] = delta
b_data['Qmin'] = qmin/base_mva
b_data['Qmax'] = qmax/base_mva
b_data['Vr'] = vmag * np.cos(delta)
b_data['Vm'] = vmag * np.sin(delta)
b_data = b_data.set_index('Bus_No')
b_data.head()

,Bus_type,Pg,Qg,Pl,Ql,Vmag,Delta,Qmin,Qmax,Vr,Vm
Bus_No,,,,,,,,,,,
1,2,0.5282,0.0,0.5635,0.114718,1.035,0.0,-0.5,0.8,1.035,0.0
2,1,0.0000,0.0,0.5041,0.103934,1.035,0.0,-0.5,0.8,1.035,0.0
3,1,0.0000,0.0,0.9342,0.192128,1.000,0.0,0.0,0.0,1.000,0.0
4,1,0.0000,0.0,0.3855,0.078077,1.000,0.0,0.0,0.0,1.000,0.0
5,1,0.0000,0.0,0.3707,0.073113,1.000,0.0,0.0,0.0,1.000,0.0


In [58]:
def d_Ir_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vm, 2) - pow(Vr, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vr, 2) - pow(Vm, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return Vm/pow(V,2)

def d_Im_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return -Vr/pow(V,2)

def I_calc(y_bus, v_df):
    ir_calc = np.zeros(len(v_df.index))
    im_calc = np.zeros(len(v_df.index))
    G = np.real(y_bus)
    B = np.imag(y_bus)
    for k  in v_df.index:
        for i in v_df.index:
            ir_calc[k-1] += (G[k-1, i-1] * v_df.loc[i, 'Vr'] - B[k-1, i-1] * v_df.loc[i, 'Vm'])
            im_calc[k-1] += (G[k-1, i-1] * v_df.loc[i, 'Vm'] + B[k-1, i-1] * v_df.loc[i, 'Vr'])
    return ir_calc, im_calc


In [85]:
def data_df(b_data_):
    b_df = b_data_
    b_df['Psh'] = b_data['Pg'] - b_data['Pl']
    b_df['Qsh'] = b_data['Qg'] - b_data['Ql']
    b_df['Ir'] = (b_df['Psh']*b_df['Vr'] + b_df['Qsh']*b_df['Vm'])/(pow(b_df['Vr'], 2) + pow(b_df['Vm'], 2))
    b_df['Im'] = (b_df['Psh']*b_df['Vm'] - b_df['Qsh']*b_df['Vr'])/(pow(b_df['Vr'], 2) + pow(b_df['Vm'], 2))
    b_df['d_Ir_Vr'] = d_Ir_Vr(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Ir_Vm'] = d_Ir_Vm(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Im_Vr'] = d_Im_Vr(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Im_Vm'] = d_Ir_Vm(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Ir_Q']  = d_Ir_Q(b_df['Vr'],b_df['Vm'])
    b_df['d_Im_Q']  = d_Im_Q(b_df['Vr'],b_df['Vm'])

    I_r_calc, I_m_calc = I_calc(Y_bus(), b_df)
    b_df['Ir_calc'] = I_r_calc
    b_df['Im_calc'] = I_m_calc

    con_pv = (b_df['Bus_type'] == 2) # Condition for PV Bus
    con_pq = (b_df['Bus_type'] == 1)
    b_df['delta_IR'] = b_df.loc[con_pq,'Ir_calc'] + b_df.loc[con_pq,'d_Ir_Vr']*b_df.loc[con_pq,'Vr'] + b_df.loc[con_pq,'d_Ir_Vm']*b_df.loc[con_pq,'Vm'] - b_df.loc[con_pq,'Ir']
    b_df['delta_IM'] = b_df.loc[con_pq,'Im_calc'] + b_df.loc[con_pq,'d_Im_Vr']*b_df.loc[con_pq,'Vr'] + b_df.loc[con_pq,'d_Im_Vm']*b_df.loc[con_pq,'Vm'] - b_df.loc[con_pq,'Im']

    b_df.loc[con_pv,'delta_IR'] = b_df.loc[con_pv,'Ir_calc'] + b_df.loc[con_pv,'d_Ir_Vm']*b_df.loc[con_pv,'Vm'] + b_df.loc[con_pv,'d_Ir_Q']*b_df.loc[con_pv,'Qsh']- b_df.loc[con_pv,'Ir']
    b_df.loc[con_pv,'delta_IM'] = b_df.loc[con_pv,'Im_calc'] + b_df.loc[con_pv,'d_Im_Vm']*b_df.loc[con_pv,'Vm'] + b_df.loc[con_pv,'d_Im_Q']*b_df.loc[con_pv,'Qsh']- b_df.loc[con_pv,'Im']

    return b_df

In [86]:
def Jacobian(df):
    b_df = data_df(df)
    delta_I = np.zeros((nbus, 1, 2, 1))
    J = np.zeros((nbus, nbus, 2, 2))
    for k in b_df.index:
        if b_df.loc[k,"Bus_type"] != 3:
            if b_df.loc[k,"Bus_type"] == 1:
                J[k-1, k-1][0, 0] = b_df.loc[k, 'd_Im_Vr']
                J[k-1, k-1][0, 1] = b_df.loc[k, 'd_Im_Vm']
                J[k-1, k-1][1, 0] = b_df.loc[k, 'd_Ir_Vr']
                J[k-1, k-1][1, 1] = b_df.loc[k, 'd_Ir_Vm']
            else:
                J[k-1, k-1][0, 0] = b_df.loc[k, 'd_Im_Vm']
                J[k-1, k-1][0, 1] = b_df.loc[k, 'd_Im_Q']
                J[k-1, k-1][1, 0] = b_df.loc[k, 'd_Ir_Vm']
                J[k-1, k-1][1, 1] = b_df.loc[k, 'd_Ir_Q']
            
            delta_I[k-1,0][0,0] = b_df.loc[k,'delta_IM']
            delta_I[k-1,0][1,0] = b_df.loc[k,'delta_IR']
    
    return delta_I, J


In [ ]:
def NR(b_data__, tol, iter):
    vr = b_data__[""]
    vm = b_data__[""]
    q = b_data__['']
    for i in range(iter):
        delta_I, J = Jacobian(b_data__)
        x = np.linalg.solve(J, delta_I)
        vr
        

In [91]:
jj = np.random.random((3,3,2,2))

In [93]:
jj.shape

(3, 3, 2, 2)

In [94]:
ii = np.random.random((3,1,2,1))

In [104]:
np.linalg.solve(jj, ii)

array([[[[-121.13336762],
         [ 120.37567731]],

        [[   1.41465526],
         [  -0.14007384]],

        [[   1.68915962],
         [  -2.22788084]]],


       [[[  -0.83893507],
         [   5.01803088]],

        [[ -21.3842821 ],
         [  10.41682091]],

        [[   0.5459366 ],
         [   0.39359386]]],


       [[[   3.66443356],
         [  -3.734887  ]],

        [[   0.55289208],
         [   0.18283004]],

        [[   0.90004417],
         [  -0.3075874 ]]]])

In [84]:
for k in b_df.index:
    print(k)
    np.linalg.inv(J[k-1,k-1])

1
2
3
4
5
6
7
8
9
10
11


LinAlgError: Singular matrix

In [63]:
b_df.to_csv('b_df.csv')

ValueError: Must pass 2-d input. shape=(73, 73, 2, 2)

(73, 73, 2, 2)

In [79]:
a = delta_I.squeeze()

In [83]:
np.squeeze(delta_I, axis=1)

array([[[ -0.26863792],
        [  0.17447664]],

       [[ -0.34664027],
        [  1.08564411]],

       [[ -0.01119799],
        [  1.82378827]],

       [[  0.13300571],
        [  0.70391904]],

       [[  0.26318697],
        [  0.64151675]],

       [[ -0.07752236],
        [  1.37894314]],

       [[ -0.50370872],
        [ -0.20061695]],

       [[  0.07284857],
        [  1.67882167]],

       [[  0.35336204],
        [  1.79297854]],

       [[ -0.02422128],
        [  2.00835075]],

       [[ -0.48765659],
        [  0.01705264]],

       [[  0.54707338],
        [ -0.101652  ]],

       [[ -0.83478151],
        [  2.77405315]],

       [[  1.18400821],
        [  0.92021372]],

       [[  1.52626936],
        [  0.92611688]],

       [[ -2.05922454],
        [ -2.06065391]],

       [[  4.76612821],
        [ -5.88503559]],

       [[ -4.1009722 ],
        [ -3.21885714]],

       [[  0.44589121],
        [ -1.07899554]],

       [[  4.32047256],
        [ -1.34511927]],

